### 智能问答系统（主文件）

在这里我们来搭建一个轻量级智能问答系统，所需要的模块，包括：
- 文本预处理：这部分已经帮大家写好，只需要看看代码就可以了。
- 搭建意图识别分类器：这部分也给大家写好了，使用fastText来做的意图识别器
- 倒排表：这部分大家需要自己去创建，同时也需要考虑相似的单词（课程视频中讲过）
- 排序：基于倒排表返回的结果，我们再根据余弦相似度来计算query跟候选问题之间的相似度，最后返回相似度最高的问题的答案。这里，我们将使用BERT来表示句子的向量。 

In [ ]:
## %env KMP_DUPLICATE_LIB_OK=TRUE  

In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import pickle
import emoji
import re
import jieba
import torch
import fasttext
from sys import platform
from torch.utils.data import DataLoader
from transformers import BertTokenizer

In [2]:
# 读取已经处理好的数据： 导入在preprocessor.ipynb中生成的data/question_answer_pares.pkl文件，并将其保存在变量QApares中
with open('data/question_answer_pares.pkl','rb') as f:
    QApares = pickle.load(f)

In [3]:
# 导入在Retrieve.ipynb中生成的data/retrieve/invertedList.pkl倒排表文件，并将其保存在变量invertedList中
with open('data/retrieve/invertedList.pkl','rb') as f:
    invertedList = pickle.load(f)

In [23]:
# 这一格的内容是从preprocessor.ipynb中粘贴而来，包含了数据预处理的几个关键函数，这部分用来处理input query string
import emoji
import re
import jieba
def clean(content):
    content = emoji.demojize(content)
    content = re.sub('<.*>','',content)
    return content
def question_cut(content):
    return list(jieba.cut(content))
def strip(wordList):
    return [word.strip() for word in wordList if word.strip()!='']
with open("data/stopWord.json","r") as f:
    stopWords = f.read().split("\n")
def rm_stop_word(wordList):
    return [word for word in wordList if word not in stopWords]

def text_processing(sentence):
    sentence = clean(sentence)
    sentence = question_cut(sentence)
    sentence = strip(sentence)
    sentence = rm_stop_word(sentence)
    return sentence

# 这一格是从Retrieve中粘贴而来，用于生成与输入数据较相近的一些候选问题的index
def get_retrieve_result(sentence):
    """
    基于输入句子，并利用倒排表返回candidate sentence ids
    """
    sentence = text_processing(sentence)
    candidate = set()
    for word in sentence:
        if word in invertedList:
            candidate = candidate | invertedList[word]
    return sentence, candidate

In [6]:
# 加载训练好的fasttext模型用于意图识别
intention = fasttext.load_model('model/fasttext.ftz')

def get_intention_result(sentence):
    '''
        输入句子，返回意图识别结果
        入参：
            sentence:输入的句子
        出参:
            fasttext_label:fasttext模型的输出，共有两种结果:__label__0和__label__1。__label__0表示闲聊型，__label__1表示任务型
    '''
    sentence = text_processing(sentence)
    sentence = ' '.join(sentence)
    fasttext_label = intention.predict(sentence)[0][0]
    return fasttext_label

In [15]:
QApares.head()

,question,answer,question_after_preprocessing
0,买二份有没有少点呀,亲亲真的不好意思我们已经是优惠价了呢小本生意请亲谅解,"[买, 二份, 有没有, 少点]"
1,那就等你们处理喽,好的亲退了,[处理]
2,那我不喜欢,颜色的话一般茶刀茶针和二合一的话都是红木檀和黑木檀哦,[喜欢]
3,不是免运费,本店茶具订单满99包邮除宁夏青海内蒙古海南新疆西藏满39包邮,"[免, 运费]"
4,好吃吗,好吃的,[好吃]


In [33]:
def doc_vec(l):
    c = 0
    total = np.zeros(300)
    for word in l:
        if word in model.vocab:
            total += model[word]
            c += 1
    if c > 0:
        return total/c
    else:
        return total

In [32]:
# 使用word2vec来得到每一个回复的doc vector (同第一次项目)
# 也可以使用bert
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format('data/retrieve/sgns.zhihu.word')
QApares['docvec'] = QApares['question_after_preprocessing'].apply(doc_vec)

In [18]:
QApares.head()

,question,answer,question_after_preprocessing,docvec
0,买二份有没有少点呀,亲亲真的不好意思我们已经是优惠价了呢小本生意请亲谅解,"[买, 二份, 有没有, 少点]","[-0.13097949884831905, 0.28594200732186437, -0..."
1,那就等你们处理喽,好的亲退了,[处理],"[0.05896899849176407, 0.4832009971141815, 0.04..."
2,那我不喜欢,颜色的话一般茶刀茶针和二合一的话都是红木檀和黑木檀哦,[喜欢],"[-0.35680198669433594, 0.4411720037460327, -0...."
3,不是免运费,本店茶具订单满99包邮除宁夏青海内蒙古海南新疆西藏满39包邮,"[免, 运费]","[-0.2459930032491684, 0.27229949831962585, -0...."
4,好吃吗,好吃的,[好吃],"[-0.44224798679351807, -0.11215300112962723, -..."


In [90]:
#导入与bert embedding相关的包，关于mxnet包下载的注意事项参考实验手册
# from bert_embedding import BertEmbedding
# import mxnet

# ctx = mxnet.cpu()
# embedding = BertEmbedding(ctx=ctx)

def cosine_similarity(a,b):
    return np.dot(a,b.T) / ( (np.dot(a,a.T) **.5) * (np.dot(b,b.T) ** .5) )

def get_best_answer(sentence, candidate):
    """
    sentence: 用户输入query， 已经处理好的
    candidate: 通过倒排表返回的候选问题的下标列表
    
    返回：最佳回复，string形式
    """
    
    ## TODO： 你需要完成这部分
    ##   计算query跟每一个候选问题之间的余弦相似度，最后排序
    ##   每个query跟候选问题首先要转化成向量形式，这里使用BERT embedding （可参考第一次项目作业）。 如果你想尝试，其他的embedding方法，也可以自行尝试如tf-idf
    
    ## 一下答案使用word2vec，bert原理一样
    sentence_word2vec = doc_vec(sentence)
    
    maxIdx = -1
    maxSimilarity = -1
    
    for c in candidate:
        vec = QApares.iloc[c]['docvec']
        similarity = cosine_similarity(sentence_word2vec, vec)
        
        if similarity > maxSimilarity:
            maxSimilarity = similarity
            maxIdx = c
    
    return QApares.iloc[c]['answer']

In [91]:
def QA(sentence):
    '''
        实现一个智能客服系统，输入一个句子sentence，返回一个回答
    '''
    # 若意图识别结果为闲聊型，则默认返回'闲聊机器人'
    if get_intention_result(sentence)=='__label__0':
        return '闲聊机器人'
    # 根据倒排表进行检索获得候选问题集
    sentence, candidate = get_retrieve_result(sentence)
    # 若候选问题集大小为0，默认返回'我不明白你在说什么'
    if len(candidate)==0:
        return '我不明白你在说什么'
    
    return get_best_answer(sentence, candidate)

In [92]:
# 测试
QA('发什么快递')

'这个看当地快递派送的呢'

In [93]:
# 测试
QA('怎么退款')

'闲聊机器人'

In [94]:
# 测试
QA('这个商品有优惠券吗')

'榴莲干没货哦'

In [95]:
# 测试
QA('一二三四五六七')

'我不明白你在说什么'